https://fbref.com/pt/equipes/e0652b02/Juventus-Estatisticas

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

In [15]:
url = "https://fbref.com/pt/equipes/e0652b02/2021-2022/all_comps/Juventus-Estatisticas-Todos-os-campeonatos"

In [16]:
site = requests.get(url)

In [17]:
site

<Response [200]>

In [18]:
bs = BeautifulSoup(site.content, "html.parser")

In [19]:
tables = bs.find_all("table")
len(tables)

56

In [20]:
for index, table in enumerate(tables):
    print(f"Tabela {index} --------- Atributos: {table.attrs}")

Tabela 0 --------- Atributos: {'class': ['stats_table', 'sortable', 'min_width'], 'id': 'stats_standard_combined', 'data-cols-to-freeze': ',1'}
Tabela 1 --------- Atributos: {'class': ['stats_table', 'sortable', 'min_width'], 'id': 'stats_standard_11222', 'data-cols-to-freeze': ',1'}
Tabela 2 --------- Atributos: {'class': ['stats_table', 'sortable', 'min_width'], 'id': 'stats_standard_11323', 'data-cols-to-freeze': ',1'}
Tabela 3 --------- Atributos: {'class': ['stats_table', 'sortable', 'min_width'], 'id': 'stats_standard_11228', 'data-cols-to-freeze': ',1'}
Tabela 4 --------- Atributos: {'class': ['stats_table', 'sortable', 'min_width'], 'id': 'stats_standard_11229', 'data-cols-to-freeze': ',1'}
Tabela 5 --------- Atributos: {'class': ['stats_table', 'sortable', 'min_width'], 'id': 'matchlogs_for', 'data-cols-to-freeze': ',1'}
Tabela 6 --------- Atributos: {'class': ['stats_table', 'sortable', 'min_width'], 'id': 'stats_keeper_combined', 'data-cols-to-freeze': ',1'}
Tabela 7 -------

In [21]:
table = bs.find("table", {"id":"stats_standard_combined"}) #re.compile

In [22]:
ano = table.find("span").get_text()

In [23]:
ano

'2021-2022 Juventus'

## Headers

Dentro de thead existem duas tr com títulos

In [29]:
columns = []
for column_name in table.thead.find_all("tr")[1]:
    result = column_name.get_text()
    if result == " ":
        continue
    columns.append(result)
    

In [30]:
columns

['Jogador',
 'Nação',
 'Pos.',
 'Idade',
 'MP',
 'Inícios',
 'Min.',
 '90s',
 'Gols',
 'Assis.',
 'G-PB',
 'PB',
 'PT',
 'CrtsA',
 'CrtV',
 'Gols',
 'Assis.',
 'G+A',
 'G-PB',
 'G+A-PB',
 'xG',
 'npxG',
 'xA',
 'npxG+xA',
 'xG',
 'xA',
 'xG+xA',
 'npxG',
 'npxG+xA',
 'Partidas']

## Dados para uma linha

Nacionalidade e Nome não estão em td

In [66]:
row = table.tbody.find_all("tr")[19]

In [75]:
nome = row.find("th").get_text()
nome

'Luca Pellegrini'

In [76]:
nac = row.find("span").get_text()[3:]
nac

'ITA'

In [84]:
result = [i.get_text() for i in row.find_all("td")[1:]] ## evitar nacionalidade vir junto, puxou separada
result

['ZG',
 '22',
 '21',
 '16',
 '1.232',
 '13.7',
 '0',
 '1',
 '0',
 '0',
 '0',
 '7',
 '0',
 '0,00',
 '0,07',
 '0,07',
 '0,00',
 '0,07',
 '0.2',
 '0.2',
 '1.6',
 '1,7',
 '0,01',
 '0,13',
 '0,14',
 '0,01',
 '0,14',
 'Partidas']

In [77]:
result.insert(0, nac)
result.insert(0, nome)

In [78]:
result

['Luca Pellegrini',
 'ITA',
 'ZG',
 '22',
 '21',
 '16',
 '1.232',
 '13.7',
 '0',
 '1',
 '0',
 '0',
 '0',
 '7',
 '0',
 '0,00',
 '0,07',
 '0,07',
 '0,00',
 '0,07',
 '0.2',
 '0.2',
 '1.6',
 '1,7',
 '0,01',
 '0,13',
 '0,14',
 '0,01',
 '0,14',
 'Partidas']

In [79]:
len(result)

30

In [80]:
len(columns)

30

Procurar em tbody

# Função

In [2]:
def get_table(html):
    bs = BeautifulSoup(html, "html.parser")
    
    table = bs.find("table", {"id":"stats_standard_combined"})
    
    columns = []
    for column_name in table.thead.find_all("tr")[1]:
        result = column_name.get_text()
        if result == " ":
            continue
        columns.append(result)
    
    df = pd.DataFrame(columns=columns)
    for row in table.tbody.find_all("tr"):
        name = row.find("th").get_text()
        nac = row.find("span").get_text()[3:]

        result = [i.get_text() for i in row.find_all("td")[1:]]

        result.insert(0,nac)
        result.insert(0,name)

        df.loc[len(df)] = result
    
    df = df.replace("", np.nan)

    df = df.dropna()

    for i in range(len(df.columns)):
        df.iloc[:,i] = df.iloc[:,i].apply(lambda x: str(x).replace(",","."))

    df["Min."] = df["Min."].apply(lambda x: str(x).replace(".",""))
    df = df.iloc[:,:15]

    df.loc[:,"Idade":"Min."] = df.loc[:,"Idade":"Min."].astype(int)
    df.loc[:,"90s"] = df.loc[:,"90s"].astype(float)
    df.loc[:,"Gols":] = df.loc[:,"Gols":].astype(int)

    df["Pos1"] = df["Pos."].str.split(".", expand=True).rename(columns={0:"Pos1",1:"Pos2"}).iloc[:,0]
    df["Pos2"] = df["Pos."].str.split(".", expand=True).rename(columns={0:"Pos1",1:"Pos2"}).iloc[:,1]

    df = df.drop("Pos.", axis = 1)

    cols_order = ['Jogador', 'Nação', 'Pos1', 'Pos2', 'Idade', 'MP', 'Inícios', 'Min.', '90s', 'Gols', 'Assis.', 'G-PB', 'PB', 'PT', 'CrtsA', 'CrtV']

    df = df[cols_order]

    df["Pos2"] = df["Pos2"].apply(lambda x: str(x).replace("None","-"))

    df["G_90"] = (df["Gols"] / df["90s"]).round(2)
    df["A_90"] = (df["Assis."] / df["90s"]).round(2)
    df["G+A_90"] = ((df["Gols"] + df["Assis."]) / df["90s"]).round(2)
    
    return df

    
    

In [257]:
df = pd.DataFrame(columns=columns)
for row in table.tbody.find_all("tr"):
    name = row.find("th").get_text()
    nac = row.find("span").get_text()[3:]
    
    result = [i.get_text() for i in row.find_all("td")[1:]]
    
    result.insert(0,nac)
    result.insert(0,name)
    
    df.loc[len(df)] = result

In [258]:
df = df.replace("", np.nan)

df = df.dropna()

for i in range(len(df.columns)):
    df.iloc[:,i] = df.iloc[:,i].apply(lambda x: str(x).replace(",","."))

df["Min."] = df["Min."].apply(lambda x: str(x).replace(".",""))
df = df.iloc[:,:15]

df.loc[:,"Idade":"Min."] = df.loc[:,"Idade":"Min."].astype(int)
df.loc[:,"90s"] = df.loc[:,"90s"].astype(float)
df.loc[:,"Gols":] = df.loc[:,"Gols":].astype(int)

df["Pos1"] = df["Pos."].str.split(".", expand=True).rename(columns={0:"Pos1",1:"Pos2"}).iloc[:,0]
df["Pos2"] = df["Pos."].str.split(".", expand=True).rename(columns={0:"Pos1",1:"Pos2"}).iloc[:,1]

df = df.drop("Pos.", axis = 1)

cols_order = ['Jogador', 'Nação', 'Pos1', 'Pos2', 'Idade', 'MP', 'Inícios', 'Min.', '90s', 'Gols', 'Assis.', 'G-PB', 'PB', 'PT', 'CrtsA', 'CrtV']

df = df[cols_order]

df["Pos2"] = df["Pos2"].apply(lambda x: str(x).replace("None","-"))

df["G_90"] = (df["Gols"] / df["90s"]).round(2)
df["A_90"] = (df["Assis."] / df["90s"]).round(2)
df["G+A_90"] = ((df["Gols"] + df["Assis."]) / df["90s"]).round(2)

In [259]:
ano

'2021-2022 Juventus'

In [260]:
df

,Jogador,Nação,Pos1,Pos2,Idade,MP,Inícios,Min.,90s,Gols,Assis.,G-PB,PB,PT,CrtsA,CrtV,G_90,A_90,G+A_90
0,Matthijs de Ligt,NED,ZG,-,21,42,40,3689,41.0,3,1,3,0,0,6,1,0.07,0.02,0.10
1,Wojciech Szczęsny,POL,G,-,31,40,40,3600,40.0,0,0,0,0,0,2,0,0.00,0.00,0.00
2,Adrien Rabiot,FRA,LT,-,26,45,39,3352,37.2,0,2,0,0,0,7,0,0.00,0.05,0.05
3,Juan Cuadrado,COL,ZG,LT,33,45,36,3299,36.7,5,4,5,0,0,11,0,0.14,0.11,0.25
4,Álvaro Morata,ESP,AT,-,28,48,35,3191,35.5,12,4,10,2,2,5,0,0.34,0.11,0.45
5,Manuel Locatelli,ITA,LT,-,23,43,33,3048,33.9,3,4,3,0,0,8,0,0.09,0.12,0.21
6,Leonardo Bonucci,ITA,ZG,-,34,34,30,2818,31.3,5,0,2,3,3,1,0,0.16,0.00,0.16
7,Alex Sandro,BRA,ZG,-,30,40,29,2811,31.2,2,1,2,0,0,4,0,0.06,0.03,0.10
8,Danilo,BRA,ZG,LT,30,31,31,2623,29.1,2,3,2,0,0,6,0,0.07,0.10,0.17
9,Paulo Dybala,ARG,AT,LT,27,39,31,2605,28.9,15,6,12,3,4,4,0,0.52,0.21,0.73


In [248]:
df["Nação"].value_counts()

ITA     13
BRA      4
FRA      2
ARG      2
NED      1
SWE      1
POR      1
BEL      1
 WAL     1
SUI      1
USA      1
URU      1
SRB      1
POL      1
ESP      1
COL      1
NOR      1
Name: Nação, dtype: int64

In [261]:
df.sort_values(by = "G+A_90", ascending = False)

,Jogador,Nação,Pos1,Pos2,Idade,MP,Inícios,Min.,90s,Gols,Assis.,G-PB,PB,PT,CrtsA,CrtV,G_90,A_90,G+A_90
9,Paulo Dybala,ARG,AT,LT,27,39,31,2605,28.9,15,6,12,3,4,4,0,0.52,0.21,0.73
13,Dušan Vlahović,SRB,AT,-,21,21,18,1636,18.2,9,1,9,0,0,2,0,0.49,0.05,0.55
17,Moise Kean,ITA,AT,-,21,42,14,1391,15.5,6,2,6,0,0,5,0,0.39,0.13,0.52
4,Álvaro Morata,ESP,AT,-,28,48,35,3191,35.5,12,4,10,2,2,5,0,0.34,0.11,0.45
20,Federico Chiesa,ITA,LT,AT,23,18,15,1197,13.3,4,2,4,0,0,0,0,0.30,0.15,0.45
22,Dejan Kulusevski,SWE,AT,LT,21,27,7,991,11.0,2,3,2,0,0,2,0,0.18,0.27,0.45
10,Federico Bernardeschi,ITA,LT,AT,27,36,23,1989,22.1,2,5,2,0,0,6,0,0.09,0.23,0.32
3,Juan Cuadrado,COL,ZG,LT,33,45,36,3299,36.7,5,4,5,0,0,11,0,0.14,0.11,0.25
23,Denis Zakaria,SUI,LT,-,24,13,9,823,9.1,1,1,1,0,0,1,0,0.11,0.11,0.22
5,Manuel Locatelli,ITA,LT,-,23,43,33,3048,33.9,3,4,3,0,0,8,0,0.09,0.12,0.21


15 ---- 2605 min
0.51------ 90 min

In [266]:
ano

'2021-2022 Juventus'

In [262]:
dic_teste = {"JUV":df}

In [270]:
dic = {"2021-2022":{"JUV":df, "INT":"inter"}
      }

In [272]:
dic["2021-2022"]["SAS"] = "Sas"

In [274]:
dic["2021-2022"]["SAS"]

'Sas'

In [271]:
dic["2021-2022"]["JUV"]

,Jogador,Nação,Pos1,Pos2,Idade,MP,Inícios,Min.,90s,Gols,Assis.,G-PB,PB,PT,CrtsA,CrtV,G_90,A_90,G+A_90
0,Matthijs de Ligt,NED,ZG,-,21,42,40,3689,41.0,3,1,3,0,0,6,1,0.07,0.02,0.10
1,Wojciech Szczęsny,POL,G,-,31,40,40,3600,40.0,0,0,0,0,0,2,0,0.00,0.00,0.00
2,Adrien Rabiot,FRA,LT,-,26,45,39,3352,37.2,0,2,0,0,0,7,0,0.00,0.05,0.05
3,Juan Cuadrado,COL,ZG,LT,33,45,36,3299,36.7,5,4,5,0,0,11,0,0.14,0.11,0.25
4,Álvaro Morata,ESP,AT,-,28,48,35,3191,35.5,12,4,10,2,2,5,0,0.34,0.11,0.45
5,Manuel Locatelli,ITA,LT,-,23,43,33,3048,33.9,3,4,3,0,0,8,0,0.09,0.12,0.21
6,Leonardo Bonucci,ITA,ZG,-,34,34,30,2818,31.3,5,0,2,3,3,1,0,0.16,0.00,0.16
7,Alex Sandro,BRA,ZG,-,30,40,29,2811,31.2,2,1,2,0,0,4,0,0.06,0.03,0.10
8,Danilo,BRA,ZG,LT,30,31,31,2623,29.1,2,3,2,0,0,6,0,0.07,0.10,0.17
9,Paulo Dybala,ARG,AT,LT,27,39,31,2605,28.9,15,6,12,3,4,4,0,0.52,0.21,0.73


In [264]:
#ano - x
#dicionario para cada time 
dic_teste["JUV"]

,Jogador,Nação,Pos1,Pos2,Idade,MP,Inícios,Min.,90s,Gols,Assis.,G-PB,PB,PT,CrtsA,CrtV,G_90,A_90,G+A_90
0,Matthijs de Ligt,NED,ZG,-,21,42,40,3689,41.0,3,1,3,0,0,6,1,0.07,0.02,0.10
1,Wojciech Szczęsny,POL,G,-,31,40,40,3600,40.0,0,0,0,0,0,2,0,0.00,0.00,0.00
2,Adrien Rabiot,FRA,LT,-,26,45,39,3352,37.2,0,2,0,0,0,7,0,0.00,0.05,0.05
3,Juan Cuadrado,COL,ZG,LT,33,45,36,3299,36.7,5,4,5,0,0,11,0,0.14,0.11,0.25
4,Álvaro Morata,ESP,AT,-,28,48,35,3191,35.5,12,4,10,2,2,5,0,0.34,0.11,0.45
5,Manuel Locatelli,ITA,LT,-,23,43,33,3048,33.9,3,4,3,0,0,8,0,0.09,0.12,0.21
6,Leonardo Bonucci,ITA,ZG,-,34,34,30,2818,31.3,5,0,2,3,3,1,0,0.16,0.00,0.16
7,Alex Sandro,BRA,ZG,-,30,40,29,2811,31.2,2,1,2,0,0,4,0,0.06,0.03,0.10
8,Danilo,BRA,ZG,LT,30,31,31,2623,29.1,2,3,2,0,0,6,0,0.07,0.10,0.17
9,Paulo Dybala,ARG,AT,LT,27,39,31,2605,28.9,15,6,12,3,4,4,0,0.52,0.21,0.73


https://fbref.com/pt/comps/11/Serie-A-Estatisticas

https://www.selenium.dev/pt-br/documentation/

In [2]:
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.common.by import By
#from selenium.webdriver import ActionChains

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
import time

from selenium.webdriver.support import expected_conditions
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.remote.command import Command

from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re

//tagName[@AttributeName="Value"]

url = "https://fbref.com/pt/comps/11/Serie-A-Estatisticas"

In [8]:
import selenium

In [9]:
selenium.__version__

'4.2.0'

In [3]:
url = "https://fbref.com/pt/comps/11/Serie-A-Estatisticas"

In [4]:
PATH = "/home/vinicius/chromedriver"
service = Service(executable_path=PATH) # selenium 4
option = Options()
option.add_argument("--disable-infobars")

https://www.soughttech.com/front/article/14409/viewArticle

https://www.selenium.dev/selenium/docs/api/py/webdriver_remote/selenium.webdriver.remote.webdriver.html

https://www.youtube.com/watch?v=AfPpdrXCPrE - Text

https://www.soughttech.com/front/article/14409/viewArticle - Soluçao att pagina

https://stackoverflow.com/questions/59507654/clicking-next-page-using-selenium

# EESE!

In [5]:
def get_table(html):
    bs = BeautifulSoup(html, "html.parser")
    
    table = bs.find("table", {"id":"stats_standard_11222"})
    
    columns = []
    for column_name in table.thead.find_all("tr")[1]:
        result = column_name.get_text()
        if result == " ":
            continue
        columns.append(result)
    
    df = pd.DataFrame(columns=columns)
    for row in table.tbody.find_all("tr"):
        name = row.find("th").get_text()
        nac = row.find("span").get_text()[3:]

        result = [i.get_text() for i in row.find_all("td")[1:]]

        result.insert(0,nac)
        result.insert(0,name)

        df.loc[len(df)] = result
    
    df = df.replace("", np.nan)

    df = df.dropna()

    for i in range(len(df.columns)):
        df.iloc[:,i] = df.iloc[:,i].apply(lambda x: str(x).replace(",","."))

    df["Min."] = df["Min."].apply(lambda x: str(x).replace(".",""))
    df = df.iloc[:,:15]

    df.loc[:,"Idade":"Min."] = df.loc[:,"Idade":"Min."].astype(int)
    df.loc[:,"90s"] = df.loc[:,"90s"].astype(float)
    df.loc[:,"Gols":] = df.loc[:,"Gols":].astype(int)

    df["Pos1"] = df["Pos."].str.split(".", expand=True).rename(columns={0:"Pos1",1:"Pos2"}).iloc[:,0]
    df["Pos2"] = df["Pos."].str.split(".", expand=True).rename(columns={0:"Pos1",1:"Pos2"}).iloc[:,1]

    df = df.drop("Pos.", axis = 1)

    cols_order = ['Jogador', 'Nação', 'Pos1', 'Pos2', 'Idade', 'MP', 'Inícios', 'Min.', '90s', 'Gols', 'Assis.', 'G-PB', 'PB', 'PT', 'CrtsA', 'CrtV']

    df = df[cols_order]

    df["Pos2"] = df["Pos2"].apply(lambda x: str(x).replace("None","-"))

    df["G_90"] = (df["Gols"] / df["90s"]).round(2)
    df["A_90"] = (df["Assis."] / df["90s"]).round(2)
    df["G+A_90"] = ((df["Gols"] + df["Assis."]) / df["90s"]).round(2)
    
    return df

    
    

In [38]:
import random
n = random.randint(0,20)

driver = webdriver.Chrome(service=service)
driver.get(url)
driver.maximize_window()
table = driver.find_element(By.ID, "results112221_overall")
teams = table.find_elements(By.XPATH, './/td[@data-stat="squad"]/a') ## o . leva em conta o elemento atual do find 
driver.execute_script("window.scrollBy(0,900)","")
time.sleep(3)
teams[3].click()


wait=WebDriverWait(driver, 10)
wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, 'aswift_2')))  #botão de fechar está dentro desse //iframe

#time.sleep(3)
ad_button = driver.find_element(By.XPATH, '//div[contains(@id, "dismiss-button")]')
#time.sleep(3)
ad_button.click()

time.sleep(3)

page = driver.page_source

#element = driver.find_element_by_xpath("//table[@id='stats_standard_11222']")
#html_content = element.get_attribute("outerHTML")

driver.quit()




In [8]:
def get_table(html):
    bs = BeautifulSoup(html, "html.parser")
    
    table = bs.find("table", {"id":"stats_standard_11222"})
    
    columns = []
    for column_name in table.thead.find_all("tr")[1]:
        result = column_name.get_text()
        if result == " ":
            continue
        columns.append(result)
    
    df = pd.DataFrame(columns=columns)
    for row in table.tbody.find_all("tr"):
        name = row.find("th").get_text()
        nac = row.find("span").get_text()[3:]

        result = [i.get_text() for i in row.find_all("td")[1:]]

        result.insert(0,nac)
        result.insert(0,name)

        df.loc[len(df)] = result
    
    df = df.replace("", np.nan)

    df = df.dropna()

    for i in range(len(df.columns)):
        df.iloc[:,i] = df.iloc[:,i].apply(lambda x: str(x).replace(",","."))

    df["Min."] = df["Min."].apply(lambda x: str(x).replace(".",""))
    df = df.iloc[:,:15]

    df.loc[:,"Idade":"Min."] = df.loc[:,"Idade":"Min."].astype(int)
    df.loc[:,"90s"] = df.loc[:,"90s"].astype(float)
    df.loc[:,"Gols":] = df.loc[:,"Gols":].astype(int)

    df["Pos1"] = df["Pos."].str.split(".", expand=True).rename(columns={0:"Pos1",1:"Pos2"}).iloc[:,0]
    df["Pos2"] = df["Pos."].str.split(".", expand=True).rename(columns={0:"Pos1",1:"Pos2"}).iloc[:,1]

    df = df.drop("Pos.", axis = 1)

    cols_order = ['Jogador', 'Nação', 'Pos1', 'Pos2', 'Idade', 'MP', 'Inícios', 'Min.', '90s', 'Gols', 'Assis.', 'G-PB', 'PB', 'PT', 'CrtsA', 'CrtV']

    df = df[cols_order]

    df["Pos2"] = df["Pos2"].apply(lambda x: str(x).replace("None","-"))

    df["G_90"] = (df["Gols"] / df["90s"]).round(2)
    df["A_90"] = (df["Assis."] / df["90s"]).round(2)
    df["G+A_90"] = ((df["Gols"] + df["Assis."]) / df["90s"]).round(2)
    
    return df

    
    

In [28]:
len(table.tbody.find_all("tr"))

35

In [ ]:
count = 0
for row in table.tbody.find_all("tr"):
    

In [39]:
bs = BeautifulSoup(page, "html.parser")

In [40]:
table = bs.find("table", {"id":"stats_standard_11222"})

In [ ]:
df = pd.DataFrame(columns=columns)
for row in table.tbody.find_all("tr"):
    name = row.find("th").get_text()
    nac = row.find("span").get_text()[3:]
    
    result = [i.get_text() for i in row.find_all("td")[1:]]
    
    result.insert(0,nac)
    result.insert(0,name)
    
    df.loc[len(df)] = result

## Parei aqui

In [ ]:
t = table.tbody.find_all("tr")[1]


In [58]:
for row in table.tbody.find_all("tr", {"class":""}):
    #if row.get("class"):
    #if row.get("class"):  ##linha que tiver classe vai pular 
     #   continue
    row_data = row.find_all("td")[1:]
    name = row.find("th").get_text()
    #try:
    #nac = row.find("span").get_text()[3:]
    #except:
        
    print(name)
    #print([i.text for i in row_data])

Wojciech Szczęsny
Matthijs de Ligt
Juan Cuadrado
Adrien Rabiot
Álvaro Morata
Paulo Dybala
Manuel Locatelli
Leonardo Bonucci
Danilo
Alex Sandro
Federico Bernardeschi
Giorgio Chiellini
Weston McKennie
Luca Pellegrini
Mattia De Sciglio
Rodrigo Bentancur
Moise Kean
Dušan Vlahović
Arthur Melo
Federico Chiesa
Daniele Rugani
Denis Zakaria
Dejan Kulusevski
Mattia Perin
Fabio Miretti
Aaron Ramsey
Kaio Jorge
Marley Aké
Carlo Pinsoglio
Cristiano Ronaldo
Martin Palumbo
Matìas Soulé
Koni De Winter
Nicolò Fagioli
Samuel Iling-Junior
Franco Israel
Marco Raina
Filippo Ranocchia
Zsombor Senkó
Diego Stramaccioni
Emanuele Zuelli


In [12]:
columns = []
for column_name in table.thead.find_all("tr")[1]:
    result = column_name.get_text()
    if result == " ":
        continue
    columns.append(result)

In [13]:
row = table.tbody.find_all("tr")[19]

In [14]:
name = row.find("th").get_text()
name

'Ștefan Radu'

In [15]:
nac = row.find("span").get_text()[3:]
nac

'ROU'

In [52]:
for i in row.find_all("td")[1:]:
    print(i.get_text())

AT,LT
24
12
8
670
7.4
0
2
0
0
0
0
0
0,00
0,27
0,27
0,00
0,27
1.0
1.0
0.8
1,8
0,13
0,10
0,24
0,13
0,24
Partidas


In [ ]:
def get_table(html):
    bs = BeautifulSoup(html, "html.parser")
    
    table = bs.find("table", {"id":"stats_standard_combined"})
    
    columns = []
    for column_name in table.thead.find_all("tr")[1]:
        result = column_name.get_text()
        if result == " ":
            continue
        columns.append(result)
    
    df = pd.DataFrame(columns=columns)
    for row in table.tbody.find_all("tr"):
        name = row.find("th").get_text()
        nac = row.find("span").get_text()[3:]

        result = [i.get_text() for i in row.find_all("td")[1:]]

        result.insert(0,nac)
        result.insert(0,name)

        df.loc[len(df)] = result
    
    df = df.replace("", np.nan)

    df = df.dropna()

    for i in range(len(df.columns)):
        df.iloc[:,i] = df.iloc[:,i].apply(lambda x: str(x).replace(",","."))

    df["Min."] = df["Min."].apply(lambda x: str(x).replace(".",""))
    df = df.iloc[:,:15]

    df.loc[:,"Idade":"Min."] = df.loc[:,"Idade":"Min."].astype(int)
    df.loc[:,"90s"] = df.loc[:,"90s"].astype(float)
    df.loc[:,"Gols":] = df.loc[:,"Gols":].astype(int)

    df["Pos1"] = df["Pos."].str.split(".", expand=True).rename(columns={0:"Pos1",1:"Pos2"}).iloc[:,0]
    df["Pos2"] = df["Pos."].str.split(".", expand=True).rename(columns={0:"Pos1",1:"Pos2"}).iloc[:,1]

    df = df.drop("Pos.", axis = 1)

    cols_order = ['Jogador', 'Nação', 'Pos1', 'Pos2', 'Idade', 'MP', 'Inícios', 'Min.', '90s', 'Gols', 'Assis.', 'G-PB', 'PB', 'PT', 'CrtsA', 'CrtV']

    df = df[cols_order]

    df["Pos2"] = df["Pos2"].apply(lambda x: str(x).replace("None","-"))

    df["G_90"] = (df["Gols"] / df["90s"]).round(2)
    df["A_90"] = (df["Assis."] / df["90s"]).round(2)
    df["G+A_90"] = ((df["Gols"] + df["Assis."]) / df["90s"]).round(2)
    
    return df

    
    

In [10]:
get_table(page)

AttributeError: 'NoneType' object has no attribute 'thead'